List Nodes in cluster

In [ ]:
%%bash 
kubectl get node

NAME    STATUS   ROLES                       AGE   VERSION
node1   Ready    control-plane,etcd,master   18d   v1.30.6+rke2r1
node2   Ready    control-plane,etcd,master   18d   v1.30.6+rke2r1
node3   Ready    control-plane,etcd,master   18d   v1.30.6+rke2r1
node4   Ready    <none>                      18d   v1.30.6+rke2r1


Get Deployments in namespace default

In [9]:
%%bash
kubectl get deployment

NAME    READY   UP-TO-DATE   AVAILABLE   AGE
nginx   1/1     1            1           89s


Create a deployment from commandline 

In [ ]:
%%bash
kubectl create deployment --image nginx:1.14.2 --port 80 nginx

Show content of deployment

In [23]:
%%bash
kubectl get deployment nginx -o yaml

apiVersion: apps/v1
kind: Deployment
metadata:
  annotations:
    deployment.kubernetes.io/revision: "2"
  creationTimestamp: "2024-11-30T11:23:19Z"
  generation: 2
  labels:
    app: nginx
  name: nginx
  namespace: default
  resourceVersion: "10146075"
  uid: 9ea09875-e518-4e76-9ad5-6ed410023908
spec:
  progressDeadlineSeconds: 600
  replicas: 1
  revisionHistoryLimit: 10
  selector:
    matchLabels:
      app: nginx
  strategy:
    rollingUpdate:
      maxSurge: 25%
      maxUnavailable: 25%
    type: RollingUpdate
  template:
    metadata:
      creationTimestamp: null
      labels:
        app: nginx
    spec:
      containers:
      - image: nginx:1.27.3
        imagePullPolicy: IfNotPresent
        name: nginx
        ports:
        - containerPort: 80
          protocol: TCP
        resources: {}
        terminationMessagePath: /dev/termination-log
        terminationMessagePolicy: File
      dnsPolicy: ClusterFirst
      restartPolicy: Always
      schedulerName: default-sched

Update image version

In [18]:
%%bash
kubectl set image deployment/nginx nginx=nginx:1.27.3

deployment.apps/nginx image updated


show all resources

In [19]:
%%bash
kubectl get all

NAME                         READY   STATUS    RESTARTS   AGE
pod/nginx-55b5bd4448-9g79d   1/1     Running   0          7m50s

NAME                 TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
service/kubernetes   ClusterIP   10.43.0.1    <none>        443/TCP   18d

NAME                    READY   UP-TO-DATE   AVAILABLE   AGE
deployment.apps/nginx   1/1     1            1           19m

NAME                               DESIRED   CURRENT   READY   AGE
replicaset.apps/nginx-55b5bd4448   1         1         1       7m51s
replicaset.apps/nginx-77d8468669   0         0         0       19m


expose nginx

In [ ]:
%%bash
kubectl expose deployment nginx --port 80 --type LoadBalancer --target-port 80 --protocol TCP

Show exposed service

In [24]:
%%bash
kubectl get service/nginx
echo "-------------------------------"
kubectl get service/nginx -o yaml

NAME    TYPE           CLUSTER-IP      EXTERNAL-IP    PORT(S)        AGE
nginx   LoadBalancer   10.43.157.146   192.168.1.10   80:31315/TCP   4m7s
-------------------------------
apiVersion: v1
kind: Service
metadata:
  annotations:
    metallb.universe.tf/ip-allocated-from-pool: picluster-pool
  creationTimestamp: "2024-11-30T11:45:16Z"
  labels:
    app: nginx
  name: nginx
  namespace: default
  resourceVersion: "10150041"
  uid: f1d4b50c-68f5-40c9-8be9-806c5dd0cbda
spec:
  allocateLoadBalancerNodePorts: true
  clusterIP: 10.43.157.146
  clusterIPs:
  - 10.43.157.146
  externalTrafficPolicy: Cluster
  internalTrafficPolicy: Cluster
  ipFamilies:
  - IPv4
  ipFamilyPolicy: SingleStack
  ports:
  - nodePort: 31315
    port: 80
    protocol: TCP
    targetPort: 80
  selector:
    app: nginx
  sessionAffinity: None
  type: LoadBalancer
status:
  loadBalancer:
    ingress:
    - ip: 192.168.1.10
      ipMode: VIP
